# Transformations XSL : les fondamentaux

## Structure d’une feuille de transformation

### Squelette

```xml
<?xml version="1.0" encoding="UTF-8"?>

<!-- root element -->
<xsl:stylesheet version="3.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

    <!-- document to be produced -->
    <xsl:output method="xml" version="1.0" encoding="utf-8" indent="yes"/>

    <!-- main rule -->
    <xsl:template match="/">
        <!-- backbone -->
    </xsl:template>

</xsl:stylesheet>
```

### Élément racine

Son rôle est de spécifier la version du langage utilisé et de déclarer l’espace de nommage qui permet de qualifier les fonctions XSL. Par convention, il s’agit du préfixe `xsl`.

### Fonction `output`

Cette fonction décrit le document de sortie à l’aide des attributs suivants :
- `method` pour le type de document (`xml` `html` `xhtml` `text` `json` `adaptive`)
- `version` pour la version du format
- `encoding` pour l’encodage utilisé
- `indent` pour un document indenté ou non (`yes` `no`)

### Rôles du template structurant

Le template structurant a pour rôle premier d’établir la règle de transformation principale en plaçant le pointeur à la racine du document à transformer. C’est le sens de l’expression XPath `/` assignée comme valeur de l’attribut `match`. Ensuite, il décrit le squelette du document de sortie en appelant les règles de transformation secondaires soit par directement par leur nom (l’attribut `name` est alors obligatoire), soit par un appel à candidatures.

## Le point sur les templates

### Définition

**Template :** modèle de transformation appliqué systématiquement (sauf contrordre).

```xml
<xsl:template match="{XPath}" name="monTpl">
    <!-- actions -->
</xsl:template>
```

### Fonction `template`

La fonction `template` décrit les actions à exécuter lorsqu’une commande lui est transmise. Il prend deux arguments :
- `match` avec une expression XPath pour sélectionner les nœuds impactés par la commande ;
- `name` (facultatif) pour lui attribuer un nom.

Un template ne produira de résultat que s’il peut répondre aux critères exigés par une commande. Considérons le code XML :

```xml
<characters>
    <character>
        <firstName>Sheldon</firstName>
        <lastName>Cooper</lastName>
    </character>
    <character>
        <firstName>Howard</firstName>
        <lastName>Hofstadter</lastName>
    </character>
</characters>
```

Si une commande spéciale `/characters` est transmise au template ci-dessous, elle ne produira aucun résultat :

```xml
<template match="firstName">
    <xsl:value-of select="."/>
</template>
```

Dans cet exemple, la commande manque de précision. Le template aurait en revanche pu répondre à une commande du type `//character`.

#### Création de nœuds

Pour créer une structure XML dans un template, il suffit souvent de recourir à une syntaxe *in extenso* :

```xml
<xsl:template match="character">
    <b lastname="lastName"><xsl:value-of select="firstName"/></b>
</xsl:template>
```

Appliqué au fragment XML décrit plus haut, ce template produira la structure suivante :

```xml
<b lastname="lastName">Sheldon</b>
<b lastname="lastName">Leonard</b>
```

Le problème ici vient du fait que l’élément `b`, n’étant pas préfixé par `xsl`, n’a aucune fonction dans le langage et se comporte comme du texte brut. Le terme `lastName` assigné comme valeur de l’attribut `lastname` n’est par conséquent interprété comme une expression XPath.

La parade consiste à soit utiliser des fonctions XSL pour créer les éléments et les attributs, soit à entourer l’expression par des accolades (`{lastName}`) dans la syntaxe *in extenso*.

```xml
<!-- curly braces to identify XPath expressions -->
<xsl:template match="character">
    <b lastname="{lastName}"><xsl:value-of select="firstName"/></b>
</xsl:template>
```

```xml
<!-- XSL functions 'element' and 'attribute' -->
<xsl:template match="character">
    <xsl:element name="b">
        <xsl:attribute name="lastname">
            <xsl:value-of select="lastName"/>
        </xsl:attribute>
        <xsl:value-of select="firstName"/>
    </xsl:element>
</xsl:template>
```

### Fonction `apply-templates`

La fonction `apply-templates` est chargée d’effectuer un appel à candidatures aux templates décrits en définissant les critères de l’appel grâce à l’attribut `select` qui prend une expression XPath comme valeur.

```xml
<!-- main rule -->
<xsl:template match="/">
    <!-- function 'element' to create '<bbt/>' -->
    <xsl:element name="bbt">
        <!-- function 'attribut' to create an attribute 'creation' -->
        <xsl:attribute name="creation">2007</xsl:attribute>
        <!-- call for application -->
        <xsl:apply-templates select="//character"/>
    </xsl:element>
</xsl:template>
```

Le template précédent donnera pour résultat :

```xml
<?xml version="1.0" encoding="UTF-8"?>
<bbt creation="2007">
   <b lastName="Cooper">Sheldon</b>
   <b lastName="Hofstadter">Leonard</b>
</bbt>
```

## Quelques opérations courantes

### Extraire l’information

#### Fonction `value-of`

Pour récupérer la valeur textuelle d’un nœud grâce à une expression XPath dans un attribut `select`.

```xml
<!-- print: Leonard -->
<xsl:value-of select="character/firstName"/>
```

#### Fonction `copy-of`

Copie en profondeur pour récupérer l’arborescence d’un nœud grâce à une expression XPath dans un attribut `select`.

```xml
<!-- print: <firstName>Leonard</firstName> -->
<xsl:copy-of select="character/firstName"/>
```

#### Fonction `text`

Pour insérer du texte.

```xml
<!-- a whitespace character -->
<xsl:text> </xsl:text>
```

### Automatiser des instructions

#### Fonction `for-each`

Pour répéter des actions sur chaque nœud renvoyé par une expression XPath validé dans un attribut `select`.

```xml
<xsl:template match="/">
    <xsl:element name="bbt">
        <xsl:attribute name="creation">2007</xsl:attribute>
        <xsl:for-each select="//character">
            <!-- print: <lastName>Hofstadter</lastName> -->
            <xsl:copy-of select="lastName"/>
        </xsl:for-each>
    </xsl:element>
</xsl:template>
```

#### Fonction `call-template`

Appel explicite à un template par son nom. L’attribut `name` prend comme valeur le nom d’un template et non pas une expression XPath.

```xml
<xsl:template match="/">
    <xsl:element name="bbt">
        <xsl:attribute name="creation">2007</xsl:attribute>
        <xsl:for-each select="//character">
            <!-- call to a specific template 'Character' -->
            <xsl:call-template name="Character"/>
        </xsl:for-each>
    </xsl:element>
</xsl:template>
```

### Trier des données

#### Fonction `sort`

Pour trier un ensemble de nœuds grâce à une expression XPath dans un attribut `select` qui désigne la clé de tri. Divers attributs permettent de gérer les paramètres.

```xml
<xsl:template match="/">
    <xsl:element name="bbt">
        <xsl:attribute name="creation">2007</xsl:attribute>
        <xsl:for-each select="//character">
            <!-- sorting by lastName -->
            <xsl:sort select="lastName"/>
            <xsl:call-template name="Character"/>
        </xsl:for-each>
    </xsl:element>
</xsl:template>
```

|Attribut|Description|Valeurs autorisées|
|:-:|:-|:-:|
|`order`|Tri croissant ou décroissant|ascending descending|
|`case-order`|Priorité aux minuscules ou aux majuscules (par défaut)|`upper-first` `lower-first`|
|`data-type`|Type des données à trier : nombres ou texte (par défaut)|`text` `number`|
|`lang`|Code langue pour le tri lexicographique|`fr` `de` `en` `ja` …|

### Tests

#### Fonction `if`

Pour émettre une condition de transformation grâce à une expression de comparaison dans un attribut `test`.

```xml
<xsl:template match="character" name="Character">
    <xsl:element name="personnage">
        <xsl:attribute name="identite">
            <xsl:value-of select="firstName"/>
            <!-- if <firstName> exists -->
            <!-- as charcater Penny doesn't have a known lastName, the whitespace is useless -->
            <xsl:if test="lastName">
                <xsl:text> </xsl:text>
                <xsl:value-of select="lastName"/>
            </xsl:if>
        </xsl:attribute>
    </xsl:element>
</xsl:template>
```

|Expression|Description|
|:-:|:-|
|`element`|Teste la présence d’un élément `element`|
|`@attribut`|Teste la présence d’un attribut `attribut`|
|`a = b`|Teste si `a` et `b` partagent la même valeur.|
|`not(a = b)`|Teste si `a` et `b` ne partagent pas la même valeur.|
|`a &lt; b`|Teste si la valeur de `a` est strictement inférieure à `b`.|
|`a &gt; b`|Teste si la valeur de `a` est strictement supérieure à `b`.|
|`a &lt;= b`|Teste si la valeur de `a` est inférieure ou égale à `b`.|
|`a &gt;= b`|Teste si la valeur de `a` est supérieure ou égale à `b`.|

**Remarques :**
- les entités doivent vraiment être encodées telles quelles ;
- les opérateurs logiques `and` et `or` permettent de combiner plusieurs conditions.

#### Fonction `choose`

Pour soumettre une transformation à des conditions successives, en combinaison des fonctions `when` et `otherwise`, la première permettant d’exprimer *x* conditions, la seconde une opération par défaut.

```xml
<!-- XML document -->
<americanDad>
    <character name="Francine" gender="F"/>
    <character name="Stan" gender="M"/>
    <character name="Roger"/>
</americanDad>
```

```xml
<!-- XSL template -->
<xsl:template match="//character">
    <xsl:choose>
        <xsl:when test="@gender = 'M'">
            <xsl:text>M. </xsl:text>
        </xsl:when>
        <xsl:when test="@gender = 'F'">
            <xsl:text>Mme. </xsl:text>
        </xsl:when>
        <xsl:otherwise>
            <xsl:text>? </xsl:text>
        </xsl:otherwise>
    </xsl:choose>
    <xsl:value-of select="@name"/>
</xsl:template>
```

```txt
<!-- print -->
Mme Francine
M. Stan
? Roger
```

## Le point sur les variables

### Document de référence

```xml
<?xml version="1.0" encoding="UTF-8"?>
<show title="The Big Bang Theory">
    <characters>
        <character pID="sheldon">
            <firstName>Sheldon</firstName>
            <lastName>Cooper</lastName>
        </character>
        <character pID="penny">
            <firstName>Penny</firstName>
        </character>
    </characters>
    <actors>
        <actor pIDREF="penny">
            <firstName>Kaley</firstName>
            <lastName>Cuoco-Sweeting</lastName>
            <birth>1985-11-30</birth>
        </actor>
        <actor pIDREF="sheldon">
            <firstName>Jim</firstName>
            <lastName>Parsons</lastName>
            <birth>1973-03-24</birth>
        </actor>
    </actors>
</show>
```

### Fonction `variable`

Pour stocker de l’information en mémoire, sélectionnée grâce à une expression XPath dans un attribut `select`. L’attribut `name` lui fournit un nom, appelé plus loin en le préfixant du signe `$`.

```xml
<xsl:template match="character" name="Character">
    <!-- 'pID' attribute stored into a variable named 'id' -->
    <xsl:variable name="id" select="@pID"/>
    <xsl:element name="character">
        <xsl:element name="id">
            <!-- reference to variable '$id' -->
            <xsl:value-of select="$id"/>
        </xsl:element>
    </xsl:element>
</xsl:template>
```

Il est possible de stocker une valeur brute dans une variable :

```xml
<xsl:variable name="show">The Big Bang Theory</xsl:variable>
```

#### Exemple

```xml
<xsl:template match="/">
    <bbt creation="2007">
        <!-- call for application -->
        <xsl:apply-templates select="//characters"/>
    </bbt>
</xsl:template>

<xsl:template match="character">
    <!-- store 'pID' value -->
    <xsl:variable name="id" select="@pID"/>
    <!-- store details about actor -->
    <xsl:variable name="actor" select="//actor[@pIDREF = $id]"/>
    <xsl:element name="character">
        <xsl:attribute name="name">
            <xsl:value-of select="firstName"/>
            <xsl:text> </xsl:text>
            <xsl:value-of select="lastName"/>
        </xsl:attribute>
        <!-- details about the actor in an attribute -->
        <xsl:attribute name="actor">
            <xsl:value-of select="$actor/firstName"/>
            <xsl:text> </xsl:text>
            <xsl:value-of select="$actor/lastName"/>
        </xsl:attribute>
    </xsl:element>
</xsl:template>
```

## Les paramètres

### Fonction `with-param`

À l’intérieur d’une fonction `call-template`, ce mécanisme permet de transmettre un paramètre au template. L’attribut `name` fait référence au paramètre et l’attribut `select` en fixe la valeur grâce à une expression XPath.

```xml
<xsl:template match="/">
    <bbt creation="2007">
        <xsl:for-each select="//character">
            <xsl:call-template name="Character">
                <!-- firstName is transmitted to the template -->
                <xsl:with-param name="printFirstName" select="firstName"/>
            </xsl:call-template>
        </xsl:for-each>
    </bbt>
</xsl:template>
```

### Fonction `param`

À l’intérieur d’une fonction template, ce mécanisme permet de déclarer un paramètre pour le template appelé. L’attribut `name` permet de le nommer.

```xml
<xsl:template match="character" name="Character">
    <!-- declaration -->
    <xsl:param name="printFirstName"/>
    <xsl:param name="labelFirstName">FirstName :</xsl:param>
    <!-- print -->
    <character>
        <xsl:value-of select="$labelFirstName"/>
        <xsl:text> </xsl:text>
        <xsl:value-of select="$printFirstName"/>
    </character>
</xsl:template>
```